##**References**

**[1]** Ryden, B. (2017). Introduction to cosmology (2nd ed.). Cambridge University Press.

#**Importe de Librerias**

In [1]:
import numpy as np

from scipy.optimize import fsolve
from scipy.integrate import odeint, solve_ivp

import plotly.graph_objects as go
from plotly.subplots import make_subplots

#**Definición de Parámetros**

In [2]:
# Scale Factor a(t0) = 1
a_0 = 1

# Hubble Parameter
H_0 = 1 # 68 km/s/Mpc

# Times
ts = np.linspace(0,100,1000)/H_0

#**Single-Component Universe**

De acuerdo con la formulación presentada por B. Ryden en su sección 5.3, el factor de escala $a(t)$, en un universo espacialmente plano, presenta una dependencia temporal en la forma de ley de potencias $a(t)\propto t^q$, donde $q(w) = 2/(3+3w)$ es un coeficiente que depende de la composición $w$ de universo a considerar. Esta solución es únicamente válida para $w\neq-1$ y con composiciones no vacias.

$$\dot{a}^2=\frac{8\pi G\varepsilon_0}{3c^2}a^{-(1+3w)}\quad\longrightarrow\quad a(t)=\left(\frac{t}{t_0}\right)^{2/(3+3w)}$$

###**1. Empty Universe**

Para un universo vacio sin componentes se sigue la solución a la ecuación de Friedmann

$$\dot{a}^2=-\frac{\kappa c^2}{R_0^2}$$

Tomando $\kappa=-1$ como un universo negativamente curvado y en expansión, la solución resulta

$$a(t)=\frac{t}{t0}\quad;\quad t_0=H_0^{-1}$$

In [3]:
t0_empty = 1/H_0

a_empty = ts/t0_empty

###**2. Matter Only**

Para una composición de materia $w=0$ y espacialmente plano $\kappa = 0$, la solución resulta

$$a(t) = \left(\frac{t}{t_0}\right)^{2/3}\quad ;\quad t_0 = \frac{2}{3}H_0^{-1}$$

In [4]:
t0_matter = 2/(3*H_0)

a_matter = (ts/t0_matter)**(2/3)

###**3. Radiation Only**

En el caso de una composición $w=1/3$ de radiación y $\kappa=0$, se tiene que

$$a(t) = \left(\frac{t}{t_0}\right)^{1/2}\quad ;\quad t_0 = \frac{1}{2}H_0^{-1}$$

In [5]:
t0_rad = 1/(2*H_0)

a_rad = (ts/t0_rad)**(1/2)

###**4. $\Lambda$ Only**

Dado que no se admite una solución $w=-1$ para composición de constante cosmológica $\varepsilon_\Lambda$, la solución se deriva de la siguiente forma

$$\dot{a}^2=\frac{8\pi G\varepsilon_\Lambda}{3c^2}a^2=H_0^2a^2\quad\longrightarrow\quad a(t)=e^{H_0(t-t_0)}\quad;\quad t_0=0$$

In [6]:
t0_lamda = 0

a_lamda = np.exp(H_0*(ts - t0_lamda))

###**5. Plotting Solution**

Visualizamos la solución a universos de única composición ilustrada en **Fig. 5.2** de la literatura **[1]**

In [7]:
go.Figure([go.Scatter(x = ts - t0_empty, y = a_empty, name = 'Empty'),
           go.Scatter(x = ts - t0_matter, y = a_matter, name = 'Matter'),
           go.Scatter(x = ts - t0_rad, y = a_rad, name = 'Radiation'),
           go.Scatter(x = ts - t0_lamda, y = a_lamda, name = r'$\Lambda$')],

          layout = go.Layout(width = 900, height = 600,
                             yaxis_range = [0, 10],
                             xaxis_range = [-2, 10],
                             xaxis_title = 'H0 (t - t0)',
                             yaxis_title = 'a (t)',
                             legend = dict(orientation = 'h', y = 1.1)))

#**Matter $+\,\Lambda$ Universe**

Para un universo espacialmente plano con composición de materia y constante cosmológica $\Lambda$, se debe satisfacer que

$$\Omega_0=1=\Omega_{m,0}+\Omega_{\Lambda,0}$$

###**$1.)\,\,\text{Big Chill}\,\,(\Omega_{m,0}=0.9)$**

En esta situación en la que $\Omega_{\Lambda,0}>0$, el destino de este universo es la expansión indefinida.

La solución se presenta como una forma trascendental del factor de escala $a(t)$. Implementamos el método `scipy.optimize.fsolve` para solucionar la forma trascendental del factor de escala como función temporalmente dependiente

$$a_{m\Lambda} = \left(\frac{\Omega_{m,0}}{1-\Omega_{m,0}}\right)^{1/3}$$
\
\
$$H_0t=\frac{2}{3\sqrt{1-\Omega_{m,0}}}\ln\left[\left(\frac{a}{a_{m\Lambda}}\right)^{3/2}+\sqrt{1+\left(\frac{a}{a_{m\Lambda}}\right)^{3/2}}\right]$$

In [8]:
Omega_m0 = 0.9

a_equality = (Omega_m0/(1 - Omega_m0))**(1/3)

t0_chill = 2/(3*H_0*np.sqrt(1 - Omega_m0))*\
           np.log((1 + np.sqrt(1 - Omega_m0))/np.sqrt(Omega_m0))

a_eq = lambda x, t: x**1.5 + (1 + x**3)**0.5 - np.exp(1.5*H_0*t*(1 - Omega_m0)**0.5)

solve_a = lambda t: fsolve(lambda a: a_eq(a/a_equality, t), t)[0]

a_chill = np.array([solve_a(t) for t in ts])

<ipython-input-8-0a4a97a69e8b>:10: RuntimeWarning:

The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.



###**$2.)\,\,\text{Big Crunch}\,\,(\Omega_{m,0}=1.1)$**

Teniendo una composición $\Omega_{\Lambda,0} < 0$, esto implica una eventual contracción y colapso del universo en el que la constante cosmológica $\Lambda$ ejerce una fuerza de atracción.

$$a_{max}=\left(\frac{\Omega_{m,0}}{1-\Omega_{m,0}}\right)^{1/3}$$
\
\
$$a(t)=a_{max}\arcsin^{2/3}\left(\frac{3H_0t}{2}\sqrt{\Omega_{m,0}-1}\right)$$



In [9]:
Omega_m0 = 1.1

a_max = (Omega_m0/(Omega_m0 - 1))**(1/3)

t0_crunch = 2/(3*H_0*np.sqrt(Omega_m0 - 1))*np.arcsin(a_max**-1.5)

a_crunch = a_max*(np.sin(1.5*H_0*ts*((Omega_m0 - 1)**0.5))**(2/3))

<ipython-input-9-b7cfe6ccf82a>:7: RuntimeWarning:

invalid value encountered in power



###**Plotting Solution**

Visualizamos los destinos de universo presentados en **Fig. 5.5** de la literatura **[1]**. A izquierda, la evolución de universos espacialmente planos con composición variada de materia. A derecha, una ampliación sobre la región del universo en contracción. Tomamos la solución previamente calculada de **Matter Only Universe** para $\Omega_{m,0}=1$

In [10]:
fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3])

fig1 = go.Figure([go.Scatter(x = ts - t0_matter, y = a_matter,
                             name = r'$\Omega_{m,0} = 1$'),
                  go.Scatter(x = ts - t0_chill, y = a_chill,
                             name = r'$\Omega_{m,0} = 0.9$'),
                  go.Scatter(x = ts[:100] - t0_crunch,
                             y = a_crunch[:100],
                             name = r'$\Omega_{m,0} = 1.1$')])

fig2 = go.Figure([go.Scatter(x = ts - t0_matter, y = a_matter,
                             showlegend = False,
                             name = r'$\Omega_{m,0} = 1$',
                             marker_color = '#636EFA'),
                  go.Scatter(x = ts - t0_chill, y = a_chill,
                             showlegend = False,
                             name = r'$\Omega_{m,0} = 0.9$',
                             marker_color = '#EF553B'),
                  go.Scatter(x = ts[:100] - t0_crunch,
                             y = a_crunch[:100],
                             showlegend = False,
                             name = r'$\Omega_{m,0} = 1.1$',
                             marker_color = '#00CC96')])

for trace in fig1.data:
    fig.add_trace(trace, row=1, col=1)

for trace in fig2.data:
    fig.add_trace(trace, row=1, col=2)

fig.update_layout(width = 900, height = 600,
                  legend = dict(orientation = 'h', y = 1.1))

fig.update_yaxes(range = [0, 50], title = 'a (t)', row = 1, col = 1)
fig.update_yaxes(range = [0, 3], title = 'a (t)', row = 1, col = 2)

fig.update_xaxes(range = [-2, 100], title = 'H0 (t - t0)', row=1, col=1)
fig.update_xaxes(range = [-1, 3], title = 'H0 (t - t0)', row = 1, col = 2)

fig.show()

#**Matter + $\Lambda$ + Curvature Universe**

A partir de las ecuaciones de Friedmann para un universo de multiple composición, particularmente **Matter + $\Lambda$** y considerando la curvatura, generamos la solución numérica de este sistema de ecuaciones diferenciales a partir de los métodos `scipy.integrate.odeint` y `scipy.integrate.solve_ivp`

$$\left(\frac{\dot{a}}{a}\right)^2=H_0^2\left(\frac{\Omega_{m,0}}{a^3}+\frac{1-\Omega_{m,0}-\Omega_{\Lambda,0}}{a^2}+\Omega_{\Lambda,0}\right)$$
\
\
$$\frac{\ddot{a}}{a}=\frac{H_0^2}{2}\left(\frac{\Omega_{m,0}}{a^3}-2\Omega_{\Lambda,0} \right)$$

In [11]:
def friedmann_acceleration(t, y, Omega_m0, Omega_Lamb0):

  a, da = y

  da = da
  d2a = -0.5*H_0*H_0*(Omega_m0/a**3 - 2*Omega_Lamb0)*a

  return np.array([da, d2a])

def friedmann(a, t, Omega_m0, Omega_Lamb0):

  da = H_0*np.sqrt(1 + Omega_m0*(1/a - 1) + Omega_Lamb0*(a**2 - 1))

  return da

###**$1.)\,\,\text{Big Chill}\,\,(\Omega_{m,0}=0.31\quad\Omega_{\Lambda,0}=0.69)$**

Este es un universo en expansión, espacialmente plano y con composición equivalente a la observada en nuestro universo.

In [12]:
Omega_m0 = 0.31
Omega_Lamb0 = 0.69

solution_forward = solve_ivp(friedmann_acceleration,
                              (0, 5), [a_0, H_0],
                              args = (Omega_m0, Omega_Lamb0),
                              method = 'LSODA')

solution_backward = solve_ivp(friedmann_acceleration,
                              (0, -5), [a_0, H_0],
                              args = (Omega_m0, Omega_Lamb0),
                              method = 'LSODA')

ts_chill = np.concatenate((solution_backward.t[::-1],
                           solution_forward.t))

as_chill = np.concatenate((solution_backward.y[0][::-1],
                           solution_forward.y[0]))

<ipython-input-11-1cacbc683375>:6: RuntimeWarning:

overflow encountered in scalar divide

/usr/local/lib/python3.11/dist-packages/scipy/integrate/_ivp/lsoda.py:161: UserWarning:

lsoda: Excess accuracy requested (tolerances too small).



###**$2.)\,\,\text{Big Crunch}\,\,(\Omega_{m,0}=0.31\quad\Omega_{\Lambda,0}=-0.31)$**

Este es un universo con curvatura negativa y eventual contracción y colapso

In [13]:
Omega_m0 = 0.31
Omega_Lamb0 = -0.31

solution_forward = solve_ivp(friedmann_acceleration,
                              (0, 5), [a_0, H_0],
                              args = (Omega_m0, Omega_Lamb0),
                              method = 'LSODA')

solution_backward = solve_ivp(friedmann_acceleration,
                              (0, -5), [a_0, H_0],
                              args = (Omega_m0, Omega_Lamb0),
                              method = 'LSODA')

ts_crunch = np.concatenate((solution_backward.t[::-1],
                           solution_forward.t))

as_crunch = np.concatenate((solution_backward.y[0][::-1],
                           solution_forward.y[0]))

<ipython-input-11-1cacbc683375>:6: RuntimeWarning:

overflow encountered in scalar divide



###**$3.)\,\,\text{Big Bounce}\,\,(\Omega_{m,0}=0.31\quad\Omega_{\Lambda,0}=1.8)$**

Universo con curvatura positiva que proviene de un estado de contracción y baja densidad, dominado por la constante cosmológica $\Lambda$, hasta un mínimo en el que la curvatura se hace dominante e inicia un proceso de expansión.

In [14]:
Omega_m0 = 0.31
Omega_Lamb0 = 1.8

solution_forward = solve_ivp(friedmann_acceleration,
                              (0, 5), [a_0, H_0],
                              args = (Omega_m0, Omega_Lamb0),
                              method = 'LSODA')

solution_backward = solve_ivp(friedmann_acceleration,
                              (0, -5), [a_0, H_0],
                              args = (Omega_m0, Omega_Lamb0),
                              method = 'LSODA')

ts_bounce = np.concatenate((solution_backward.t[::-1],
                           solution_forward.t))

as_bounce = np.concatenate((solution_backward.y[0][::-1],
                           solution_forward.y[0]))

###**$4.)\,\,\text{Loitering Universe}\,\,(\Omega_{m,0}=0.31\quad\Omega_{\Lambda,0}=1.7289)$**

Este tipo de universo proviene de un estado de expansión dominado por la materia e ingresa a un estado quasi-estatico en el que el factor de escala $a(t)$ se mantiene relativamente constante durante un largo periodo de tiempo. Eventualmente, los efectos de la constante cosmológica se hacen dominantes y la expansión retorna de forma exponencial

In [15]:
Omega_m0 = 0.31
Omega_Lamb0 = 1.7289

solution_forward = odeint(friedmann, y0 = a_0,
                          t = np.linspace(0, 6, 100),
                          args = (Omega_m0, Omega_Lamb0))

solution_backward = odeint(friedmann, y0 = a_0,
                           t = np.linspace(0, -6, 100),
                           args = (Omega_m0, Omega_Lamb0))

ts_loterie = np.concatenate((np.linspace(-6, 0, 100),
                             np.linspace(0, 6, 100)))

as_loterie = np.concatenate((solution_backward.T[0][::-1],
                             solution_forward.T[0]))

<ipython-input-11-1cacbc683375>:12: RuntimeWarning:

invalid value encountered in sqrt

<ipython-input-15-b626dacf504e>:8: ODEintWarning:

Excess accuracy requested (tolerances too small). Run with full_output = 1 to get quantitative information.



###**Plotting Solution**

Visualizamos la solución a los universos anteriormente mencionados, y presentados en **Fig. 5.7** de la literatura **[1]**

In [16]:
go.Figure([go.Scatter(x = ts_chill,
                      y = as_chill,
                      name = r'$\Omega_{\Lambda_0}=0.69$'),
           go.Scatter(x = ts_crunch,
                      y = as_crunch,
                      name = r'$\Omega_{\Lambda_0}=-0.31$'),
           go.Scatter(x = ts_bounce,
                      y = as_bounce,
                      name = r'$\Omega_{\Lambda_0}=1.8$'),
           go.Scatter(x = ts_loterie,
                      y = as_loterie,
                      name = r'$\Omega_{\Lambda_0}=1.7289$')],

          layout = go.Layout(width = 900, height = 600,
                             xaxis_title = 'H0 (t - t0)',
                             yaxis_title = 'a (t)',
                             legend = dict(orientation = 'h', y = 1.1),
                             yaxis_range = [0, 3]))